In [ ]:
!pip install autoscraper

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

In [2]:
import time
from tqdm import tqdm

In [3]:
driver = webdriver.Chrome()

open a particular page using the driver

In [6]:
driver.get("https://www.amazon.in/s?k=laptops&crid=33WZPHH4FTS8&sprefix=laptops%2Caps%2C217&ref=nb_sb_noss_2")

now that you opened the webpage, now its time to to extract the data from the webpage, for that we gonna use the BeautifulSoup library

driver.page_source will give complete page source

In [7]:
html_data = BeautifulSoup(driver.page_source, 'html.parser')
# html_data

In [8]:
no_of_pages = int(html_data.find('span', {'class': 's-pagination-item s-pagination-disabled'}).text)
no_of_pages

20

now that we get to now total number of product pages, lets move into to the next step and i.e.. to extract basic information like product name, price & rating

In [13]:
prod_name_list = []
prod_img_list = []
prod_price_list = []

In [15]:
for pg_num in tqdm(range(1, no_of_pages+1)):
    url = f"https://www.amazon.in/s?k=laptops&crid=33WZPHH4FTS8&sprefix=laptops%2Caps%2C217&ref=nb_sb_noss_2&page={pg_num}"
    driver.get(url)
    time.sleep(10)
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    products = html_data.find_all('div', {'data-component-type':'s-search-result'})
    for product in products:
        product_name = product.find('h2', {'class':'a-size-medium a-spacing-none a-color-base a-text-normal'}).text
        prod_name_list.append(product_name)
        image = product.find('img')['src']
        prod_img_list.append(image)
        # rating = product.find('span', {'class':'a-icon-alt'})
        # if rating == 'None':
        #     rating = 'No rating'
        # else:
        #     rating = product.find('span', {'class':'a-icon-alt'}).text        
        # prod_rating_list.append(rating)
        price_element = product.find('span', {'class': 'a-price-whole'}) #Find the price element
        price = 'Price not available'
        if price_element:
            price = '₹' + price_element.text.strip() #If the element is found, extract the price. Added strip() to remove extra whitespaces

        prod_price_list.append(price)

100%|██████████| 20/20 [04:13<00:00, 12.67s/it]


In [16]:
len(prod_price_list)

422

In [17]:
len(prod_name_list)

423

In [19]:
len(prod_img_list)

423

In [20]:
import pandas as pd

# Ensure all lists are of the same length by truncating the longer lists
min_length = min(len(prod_name_list), len(prod_img_list), len(prod_price_list))

prod_name_list = prod_name_list[:min_length]
prod_img_list = prod_img_list[:min_length]
prod_price_list = prod_price_list[:min_length]

# Create the DataFrame
products_df = pd.DataFrame({
    'Product Name': prod_name_list,
    'Product Image': prod_img_list,
    'Product Price': prod_price_list
})

products_df

,Product Name,Product Image,Product Price
0,"Dell Inspiron 3520 Laptop, 12th Gen Intel Core...",https://m.media-amazon.com/images/I/51QGlg5iRd...,"₹47,450"
1,Lenovo IdeaPad Slim 3 12th Gen Intel Core i5-1...,https://m.media-amazon.com/images/I/81mgUtc9iK...,"₹53,560"
2,(Refurbished) Lenovo ThinkPad 8th Gen Intel Co...,https://m.media-amazon.com/images/I/611AaVzaCQ...,"₹17,765"
3,(Refurbished) HP EliteBook 820 G4 7th Gen Inte...,https://m.media-amazon.com/images/I/61HoUL4Chj...,"₹15,449"
4,"HP Laptop 255 G9 (2024), AMD Ryzen 5 5625U Hex...",https://m.media-amazon.com/images/I/71NfES5o1W...,"₹30,290"
...,...,...,...
417,(Renewed) DELL Latitude 5490 Core i5 8th Gen L...,https://m.media-amazon.com/images/I/615te9wVfz...,"₹32,990"
418,"ASUS Vivobook Go 15 (2023), Intel Core i3-N305...",https://m.media-amazon.com/images/I/71GbvoTwsQ...,"₹22,950"
419,"Lenovo V15 Intel Celeron N4500 15.6"" (39.62 cm...",https://m.media-amazon.com/images/I/516btukUor...,"₹22,990"
420,(Refurbished) HP Workstation Zbook Intel Core ...,https://m.media-amazon.com/images/I/61QK+kGGcX...,"₹49,499"


In [21]:
products_df.to_csv('amazon_products.csv', index=False)